In [2]:
import os

sample_futures_strategy_nm = "multi_period_momentum"

os.makedirs("/Users/seongmin/Desktop/neomatrix_trading", exist_ok=True)
os.makedirs("/Users/seongmin/Desktop/neomatrix_trading", exist_ok=True)
os.makedirs("/Users/seongmin/Desktop/neomatrix_trading/module/futures/", exist_ok=True)
os.makedirs(f"/Users/seongmin/Desktop/neomatrix_trading/futures/{sample_futures_strategy_nm}", exist_ok=True)

# system config
!wget https://raw.githubusercontent.com/NeoMatrixAI/strategy/main/futures/futures_config.py -O /Users/seongmin/Desktop/neomatrix_trading/futures/futures_config.py

# sample strategy - multi_period_momentum
!wget https://raw.githubusercontent.com/NeoMatrixAI/strategy/main/futures/multi_period_momentum/multi_period_momentum.py -O /Users/seongmin/Desktop/neomatrix_trading/futures/multi_period_momentum/multi_period_momentum.py
!wget https://raw.githubusercontent.com/NeoMatrixAI/strategy/main/futures/multi_period_momentum/multi_period_momentum_config.py -O /Users/seongmin/Desktop/neomatrix_trading/futures/multi_period_momentum/multi_period_momentum_config.py

# module
!wget https://raw.githubusercontent.com/NeoMatrixAI/nb-runner/main/module/log_viewer.py -O /Users/seongmin/Desktop/neomatrix_trading/module/log_viewer.py  # For real-time-trade logging
!wget https://raw.githubusercontent.com/NeoMatrixAI/nb-runner/main/module/futures/position.py -O /Users/seongmin/Desktop/neomatrix_trading/module/futures/position.py  # position
!wget https://raw.githubusercontent.com/NeoMatrixAI/nb-runner/main/module/futures/trade.py -O /Users/seongmin/Desktop/neomatrix_trading/module/futures/trade.py  # trade

--2025-09-27 12:54:24--  https://raw.githubusercontent.com/NeoMatrixAI/strategy/main/futures/futures_config.py
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 616 [text/plain]
저장 위치: `/Users/seongmin/Desktop/neomatrix_trading/futures/futures_config.py'

/Users/seongmin/Des 100%[===================>]     616  --.-KB/s    /  0s      

2025-09-27 12:54:24 (36.7 MB/s) - `/Users/seongmin/Desktop/neomatrix_trading/futures/futures_config.py' 저장함 [616/616]

--2025-09-27 12:54:25--  https://raw.githubusercontent.com/NeoMatrixAI/strategy/main/futures/multi_period_momentum/multi_period_momentum.py
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:4

In [3]:
import os

base_dir = "/Users/seongmin/Desktop/neomatrix_trading"

# List of files to check
files_to_check = [
    f"{base_dir}/module/log_viewer.py",
    f"{base_dir}/module/futures/position.py",
    f"{base_dir}/module/futures/trade.py",
    f"{base_dir}/futures/futures_config.py",
    f"{base_dir}/futures/multi_period_momentum/multi_period_momentum.py",
    f"{base_dir}/futures/multi_period_momentum/multi_period_momentum_config.py",
]

# Check if files exist
print("🔍 Checking downloaded files:")
for f in files_to_check:
    if os.path.exists(f):
        print(f"✅ Found: {f}")
    else:
        print(f"❌ Missing: {f}")

🔍 Checking downloaded files:
✅ Found: /Users/seongmin/Desktop/neomatrix_trading/module/log_viewer.py
✅ Found: /Users/seongmin/Desktop/neomatrix_trading/module/futures/position.py
✅ Found: /Users/seongmin/Desktop/neomatrix_trading/module/futures/trade.py
✅ Found: /Users/seongmin/Desktop/neomatrix_trading/futures/futures_config.py
✅ Found: /Users/seongmin/Desktop/neomatrix_trading/futures/multi_period_momentum/multi_period_momentum.py
✅ Found: /Users/seongmin/Desktop/neomatrix_trading/futures/multi_period_momentum/multi_period_momentum_config.py


In [19]:
# Create Fake module
import sys
import types
from abc import ABC, abstractmethod
from datetime import datetime, timedelta
import requests
import pandas as pd

# 1. Create a fake module called 'module' and register it with the system.
module_obj = types.ModuleType('module')
sys.modules['module'] = module_obj

# 2. Create a fake empty module called 'module.data_context' and register it with the system.
data_context_module = types.ModuleType('module.data_context')
sys.modules['module.data_context'] = data_context_module

# 3. Define the DataContext class required by the strategy.
class DataContext(ABC):
    @property
    @abstractmethod
    def current_dt(self) -> datetime: pass
    @abstractmethod
    def get_history(self, assets: list, window: int, frequency: str, fields: str or list = 'close') -> pd.DataFrame: pass

# 4. Place the defined DataContext class inside the fake 'module.data_context' module.
data_context_module.DataContext = DataContext

print("✅ Fake 'module.data_context' module and DataContext class created successfully.")

✅ Fake 'module.data_context' module and DataContext class created successfully.


In [21]:
# 1. Import example strategy, config
from futures.multi_period_momentum.multi_period_momentum import strategy     # Replace with your own strategy file
import futures.multi_period_momentum.multi_period_momentum_config as config  # Replace with your own config file

# 2. Prepare mock objects and data required for testing.
class MockDataContext(DataContext):
    def __init__(self, fake_data):
        self._fake_data = fake_data

    @property
    def current_dt(self) -> datetime:
        return datetime.now()

    def get_history(self, assets, window, frequency, fields):
        return self._fake_data[fields]

def create_fake_data(assets, num_periods):
    dates = pd.date_range(end=datetime.now(), periods=num_periods, freq='T')
    all_dfs = []
    for i, asset in enumerate(assets):
        trend = 1 + (i - len(assets) / 2) * 0.0001
        prices = [100 * (trend ** j) for j in range(num_periods)]
        df = pd.DataFrame({'close': prices}, index=dates)
        df['asset'] = asset
        all_dfs.append(df)
    df_concat = pd.concat(all_dfs).reset_index().rename(columns={'index': 'datetime'})
    df_final = df_concat.set_index(['asset', 'datetime'])
    display(df_final)
    return df_final

# 3. Run verify test
test_config = {
    "strategy_config": config.strategy_config
}
fake_data = create_fake_data(
    assets=test_config["strategy_config"]["assets"], num_periods=1440)
mock_context = MockDataContext(fake_data)

print("\n🚀 Starting strategy function testing...")
weights = strategy(context=mock_context, config_dict=test_config)
print("✅ Strategy function execution complete!")
print("\n[Final weighted result]")
weights

/var/folders/3r/df6gd3t56pq6t10_rqr2h7pr0000gn/T/ipykernel_92858/958222562.py:18: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates = pd.date_range(end=datetime.now(), periods=num_periods, freq='T')


close
asset    datetime                              
BTCUSDT  2025-09-28 17:38:29.174168  100.000000
         2025-09-28 17:39:29.174168   99.950000
         2025-09-28 17:40:29.174168   99.900025
         2025-09-28 17:41:29.174168   99.850075
         2025-09-28 17:42:29.174168   99.800150
...                                         ...
DOGEUSDT 2025-09-29 17:33:29.174168  177.515054
         2025-09-29 17:34:29.174168  177.586060
         2025-09-29 17:35:29.174168  177.657094
         2025-09-29 17:36:29.174168  177.728157
         2025-09-29 17:37:29.174168  177.799249

[14400 rows x 1 columns]


🚀 Starting strategy function testing...
✅ Strategy function execution complete!

[Final weighted result]


{'DOGEUSDT': np.float64(0.28196479124629176),
 'DOTUSDT': np.float64(0.2099924907769153),
 'ETCUSDT': np.float64(0.06902508951052684),
 'TRXUSDT': np.float64(0.13901762846626597),
 'BCHUSDT': np.float64(-0.0404595919624356),
 'BTCUSDT': np.float64(-0.099084963725916),
 'ETHUSDT': np.float64(-0.07981283159901574),
 'LTCUSDT': np.float64(-0.02037020242560996),
 'XRPUSDT': np.float64(-0.06027241028702267)}

In [22]:
mock_context

In [47]:
from futures.momentum_with_vol.momentum_with_vol import strategy
import futures.momentum_with_vol.momentum_with_vol_config as config

# 2. Prepare mock objects and data required for testing.
class MockDataContext(DataContext):
    def __init__(self, fake_data):
        self._fake_data = fake_data

    @property
    def current_dt(self) -> datetime:
        return datetime.now()

    def get_history(self, assets, window, frequency, fields):
        return self._fake_data[fields]

def create_fake_data(assets, num_periods):
    dates = pd.date_range(end=datetime.now(), periods=num_periods, freq='T')
    all_dfs = []
    for i, asset in enumerate(assets):
        trend = 1 + (i - len(assets) / 2) * 0.0001
        prices = [100 * (trend ** j) for j in range(num_periods)]
        df = pd.DataFrame({'close': prices, 'volume':prices}, index=dates)
        df['asset'] = asset
        all_dfs.append(df)
    df_concat = pd.concat(all_dfs).reset_index().rename(columns={'index': 'datetime'})
    df_final = df_concat.set_index(['asset', 'datetime'])
    display(df_final)
    return df_final
    
test_config = {
    "strategy_config": config.strategy_config
}
fake_data = create_fake_data(
    assets=test_config["strategy_config"]["assets"], num_periods=1440)
mock_context = MockDataContext(fake_data)

print("\n🚀 Starting strategy function testing...")
weights = strategy(context=mock_context, config_dict=test_config)
print("✅ Strategy function execution complete!")
print("\n[Final weighted result]")
weights

/var/folders/3r/df6gd3t56pq6t10_rqr2h7pr0000gn/T/ipykernel_82599/3306429850.py:17: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates = pd.date_range(end=datetime.now(), periods=num_periods, freq='T')


close      volume
asset    datetime                                          
BTCUSDT  2025-09-27 21:57:38.226180  100.000000  100.000000
         2025-09-27 21:58:38.226180   99.950000   99.950000
         2025-09-27 21:59:38.226180   99.900025   99.900025
         2025-09-27 22:00:38.226180   99.850075   99.850075
         2025-09-27 22:01:38.226180   99.800150   99.800150
...                                         ...         ...
DOGEUSDT 2025-09-28 21:52:38.226180  177.515054  177.515054
         2025-09-28 21:53:38.226180  177.586060  177.586060
         2025-09-28 21:54:38.226180  177.657094  177.657094
         2025-09-28 21:55:38.226180  177.728157  177.728157
         2025-09-28 21:56:38.226180  177.799249  177.799249

[14400 rows x 2 columns]


🚀 Starting strategy function testing...
✅ Strategy function execution complete!

[Final weighted result]


{}

# Backtesting

In [106]:
USER_KEY = "cc3948f0cd364a9f684a1fe78b9229ba"      #  Input your User key
DATA_KEY = "a71eaf04-802f-40be-93c2-5bee2548f4db"  #  Fixed Data Api key

print("USER_KEY:", USER_KEY if USER_KEY else "Not found")
print("DATA_KEY:", DATA_KEY if DATA_KEY else "Not found")

USER_KEY: cc3948f0cd364a9f684a1fe78b9229ba
DATA_KEY: a71eaf04-802f-40be-93c2-5bee2548f4db


In [107]:
import requests

root_url = f'https://zipline.fin.cloud.ainode.ai/'
headers={"API-KEY": USER_KEY}
requests.get(root_url, headers=headers).json()

{'message': 'API Server is Running.', 'status': 200, 'User status': 'active'}

In [108]:
import os
strategy_name = "anomarly_vol"

In [109]:
# Upload or Update strategy file
endpoint = 'upload/strategy'
url = root_url + endpoint

base_path = "/Users/seongmin/Desktop/neomatrix_trading/futures"
file_path = os.path.join(base_path, strategy_name, strategy_name + ".py")

params = {'tradeType':'futures', 'overwrite': True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'anomarly_vol' Strategy Updated Successfully!"}


In [110]:
# Check strategy file upload
endpoint = 'upload/check/strategy'
url = root_url + endpoint

params = {"tradeType": "futures", "strategy_name": strategy_name}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# strategy.py
import pandas as pd
import numpy as np
from module.data_context import DataContext # Import the necessary module

def strategy(context: DataContext, config_dict: dict) -> dict:
    """
    Implements the Micro-structure Volatility Anomaly strategy with position limits.

    This version correctly fetches historical data using the DataContext module before
    applying the trading logic.
    - It fetches data for a pre-defined asset list.
    - It selects the top N long and top N short candidates based on the signal.
    - It shorts assets with unusually high recent volatility (mean-reversion).
    - It longs assets with unusually low recent volatility.
    """
    # --- 1. Configuration Extraction ---
    strategy_params = config_dict.get("strategy_config", {})
    assets = strategy_params.get("assets", [])
    short_vol_window = strategy_params.get("short_vol_window", 20)
    long_vol_window = strategy_params.get("long_vol_window", 60)
    clip_threshold = strategy_param

In [111]:
configs = {
    "rebalancing_config": {
        "rebalancing_interval_hours": 24, # 6,12,24,72
    },
    "strategy_config":
    { "assets": ['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'BNBUSDT', 'XRPUSDT',
                        'TONUSDT', 'DOGEUSDT', 'ADAUSDT', 'TRXUSDT', 'AVAXUSDT',
                        'DOTUSDT', 'BCHUSDT', 'LTCUSDT', 'NEARUSDT', 'KASUSDT',
                        'ICPUSDT', 'XLMUSDT', 'ETCUSDT', 'APTUSDT', 'FETUSDT',
                        'FILUSDT', 'STXUSDT', 'ATOMUSDT', 'HBARUSDT', 'VETUSDT',
                        'SUIUSDT', 'ARUSDT', 'THETAUSDT', 'RUNEUSDT', 'ALGOUSDT',
                        'COREUSDT', 'FLOWUSDT', 'BSVUSDT', 'SEIUSDT', 'EGLDUSDT',
                        'XTZUSDT', 'NEOUSDT', 'CFXUSDT', 'MINAUSDT', 'IOTAUSDT',
                        'LUNCUSDT', 'CKBUSDT', 'IOTXUSDT', 'KSMUSDT', 'DYMUSDT',
                        'ZILUSDT', 'CELOUSDT', 'QTUMUSDT', 'RVNUSDT', 'ONEUSDT',
                        'ONTUSDT', 'ICXUSDT', 'BBUSDT', 'COTIUSDT', 'SXPUSDT',
                        'LSKUSDT', 'NTRNUSDT', 'TAIKOUSDT', 'WAXPUSDT', 'IOSTUSDT',
                        'XVGUSDT', 'NKNUSDT', 'BEAMUSDT', 'MEMEUSDT', 'JUPUSDT',
                        'ARBUSDT', 'GTCUSDT'],
        # Defines the lookback period (in minutes) for calculating short-term volatility.
        # This parameter captures the most recent price fluctuation intensity.
        "short_vol_window": 20,

        # Defines the lookback period (in minutes) for the medium-term volatility benchmark.
        # This acts as the baseline to which the short-term volatility is compared.
        "long_vol_window": 60,

        # Sets the maximum absolute value for the raw signal before normalization.
        # This helps to mitigate the impact of extreme volatility spikes on position sizing.
        "clip_threshold": 2.0
    }
}

In [112]:
# System configs
start_date_str = "2025-09-01 09:00"
end_date_str = "2025-09-20 08:59"
lookback_min = 360 # Max lookback minutes the script needs for data history
initial_capital = 10000
leverage = 10
generate_report_flag = True

In [113]:
import json
from datetime import datetime

path = "/Users/seongmin/Desktop/neomatrix_trading/backtest_report"

request_payload = {
    "data_apikey": DATA_KEY,
    "strategy": strategy_name,
    "strategy_config": configs,
    "start_date": start_date_str,
    "end_date": end_date_str,
    "lookback_minutes": lookback_min,
    "capital": initial_capital,
    "leverage": leverage,
    "calendar": "24/7",
    "frequency": "minute",

    "generate_pyfolio_report": generate_report_flag
}

endpoint = '/run/futures/backtest'
url = root_url + endpoint

try:
    response = requests.post(url, json=request_payload, headers=headers)
    response.raise_for_status()
    print(f"\n--- Backtest execution successful (Status Code: {response.status_code}) ---")

    try:
        result_data = response.json()
        report_type = result_data.get('report_type') # Check report type

        if report_type == 'html':
            print("Report Type: HTML Report included.")
            html_content = result_data.get('html_content')
            logs = result_data.get('logs', 'No stderr logs received.')
            stdout_logs = result_data.get('stdout', 'No stdout received.')

            if html_content:
                if not os.path.exists(path):
                    os.makedirs(path)

                # Set the path to save backtest_report.html
                # Below are the default path
                report_filename = os.path.join(path, f"{datetime.now().strftime('%Y-%m-%d %H:%M')}_{strategy_name}_backtest_report.html")
                try:
                    with open(report_filename, "w", encoding="utf-8") as f:
                        f.write(html_content)
                    print(f"HTML report received and saved successfully as '{report_filename}'.")
                except Exception as e:
                    print(f"ERROR: Failed to save received HTML report: {e}")
                    print("\n--- Received HTML Content (Snippet) ---")
                    print(html_content[:1000] + "...") # Output some content when saving fails
            else:
                print("WARN: Report type was 'html' but no HTML content found in response.")

            # print logging
            print("\n--- Execution Logs (stderr) ---")
            print(logs)
            if stdout_logs:
                print("\n--- Execution Output (stdout) ---")
                print(stdout_logs)

        elif report_type == 'logs_only':
            print(f"Report Type: Logs Only (Report generation skipped).")
            print(f"Message: {result_data.get('message')}")
            print("\n--- Execution Logs (stderr) ---")
            print(result_data.get('logs', 'No stderr logs received.'))
            if 'stdout' in result_data:
                 print("\n--- Execution Output (stdout) ---")
                 print(result_data.get('stdout', 'No stdout received.'))
        else:
            print(f"WARN: Received successful response with unknown report_type: '{report_type}'")
            print("\n--- Full JSON Response ---")
            print(json.dumps(result_data, indent=2, ensure_ascii=False))

    except json.JSONDecodeError:
        print("ERROR: Failed to decode JSON response from successful API call.")
        print(f"Content-Type: {response.headers.get('content-type', 'N/A')}")
        print("\n--- Received Raw Content (First 1000 chars) ---")
        print(response.text[:1000] + "...")

except requests.exceptions.Timeout:
    print(f"\n--- API call failed: Timeout) ---")
except requests.exceptions.HTTPError as e:
    print(f"\n--- API call failed: HTTP Error {e.response.status_code} ---")
    try:
        error_details = e.response.json()
        print("Error details:")
        print(json.dumps(error_details, indent=2, ensure_ascii=False))
    except json.JSONDecodeError:
        print("Error response content (Non-JSON):")
        print(e.response.text)
except requests.exceptions.RequestException as e:
    print(f"\n--- API call failure: Request Error ---")
    print(f"Error connecting to or requesting the API server ({url}): {e}")
except Exception as e:
    print(f"\n--- Unexpected error occurred ---")
    print(f"Error type: {type(e).__name__}, Content: {e}")


--- Backtest execution successful (Status Code: 200) ---
Report Type: HTML Report included.
HTML report received and saved successfully as '/Users/seongmin/Desktop/neomatrix_trading/backtest_report/2025-09-30 08:51_anomarly_vol_backtest_report.html'.

--- Execution Logs (stderr) ---
[2025-09-29 23:49:38+0000] Backtest script execution started.
INFO: Command line arguments parsed.

--- Runtime Configuration ---
  - user_key: Provided (Masked)
  - data_apikey: Provided (Masked)
  - strategy_config_json: Provided (Number of parameters: 20)
  - start_date: 2025-09-01 09:00
  - end_date: 2025-09-20 08:59
  - lookback_minutes: 360
  - capital: 10000.0
  - leverage: 10.0
  - calendar: 24/7
  - frequency: minute
  - output_file: /app/backtest/temp_backtest_results/pyfolio_report_c57c3ce3-b708-49b3-bc67-e186eb23c0f2.html
  - generate_report: True
-----------------------------
INFO: Loading dynamic strategy and parsing config...
INFO: Dynamic strategy loaded successfully from 'anomarly_vol.py'.

# Trading

In [114]:
USER_KEY = "cc3948f0cd364a9f684a1fe78b9229ba"      #  Input your User key
DATA_KEY = "a71eaf04-802f-40be-93c2-5bee2548f4db"  #  Fixed Data Api key

print("USER_KEY:", USER_KEY if USER_KEY else "Not found")
print("DATA_KEY:", DATA_KEY if DATA_KEY else "Not found")

import requests

root_url = f'https://aifapbt.fin.cloud.ainode.ai/'

headers = {"API-KEY": USER_KEY}
requests.get(root_url, headers=headers).json()

import os
strategy_name = "anomarly_vol"

USER_KEY: cc3948f0cd364a9f684a1fe78b9229ba
DATA_KEY: a71eaf04-802f-40be-93c2-5bee2548f4db


In [115]:
# Upload or Update strategy file
endpoint = 'upload/strategy'
url = root_url + endpoint

base_path = "/Users/seongmin/Desktop/neomatrix_trading/futures"
file_path = os.path.join(base_path, strategy_name, strategy_name + ".py")

params = {'tradeType':'futures', 'overwrite': True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'anomarly_vol' Strategy Updated Successfully!"}


In [116]:
# Check strategy file upload
endpoint = 'upload/check/strategy'
url = root_url + endpoint

params = {"tradeType": "futures", "strategy_name": strategy_name}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# strategy.py
import pandas as pd
import numpy as np
from module.data_context import DataContext # Import the necessary module

def strategy(context: DataContext, config_dict: dict) -> dict:
    """
    Implements the Micro-structure Volatility Anomaly strategy with position limits.

    This version correctly fetches historical data using the DataContext module before
    applying the trading logic.
    - It fetches data for a pre-defined asset list.
    - It selects the top N long and top N short candidates based on the signal.
    - It shorts assets with unusually high recent volatility (mean-reversion).
    - It longs assets with unusually low recent volatility.
    """
    # --- 1. Configuration Extraction ---
    strategy_params = config_dict.get("strategy_config", {})
    assets = strategy_params.get("assets", [])
    short_vol_window = strategy_params.get("short_vol_window", 20)
    long_vol_window = strategy_params.get("long_vol_window", 60)
    clip_threshold = strategy_param

In [117]:
# Upload or Update strategy-config file
endpoint = 'upload/strategy-config'
url = root_url + endpoint

strategy_config_name = strategy_name + "_config"
base_path = "/Users/seongmin/Desktop/neomatrix_trading/futures/"
file_path = os.path.join(base_path, strategy_name, strategy_config_name + ".py")

params = {"tradeType": "futures", "overwrite": True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'anomarly_vol' Strategy-Config Updated Successfully!"}


In [118]:
# Check strategy-config file upload
endpoint = 'upload/check/strategy-config'
url = root_url + endpoint

params = {"tradeType": "futures", "strategy_name": strategy_name}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# strategy_config.py

strategy_config = {
    "assets": ['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'BNBUSDT', 'XRPUSDT',
                        'TONUSDT', 'DOGEUSDT', 'ADAUSDT', 'TRXUSDT', 'AVAXUSDT',
                        'DOTUSDT', 'BCHUSDT', 'LTCUSDT', 'NEARUSDT', 'KASUSDT',
                        'ICPUSDT', 'XLMUSDT', 'ETCUSDT', 'APTUSDT', 'FETUSDT',
                        'FILUSDT', 'STXUSDT', 'ATOMUSDT', 'HBARUSDT', 'VETUSDT',
                        'SUIUSDT', 'ARUSDT', 'THETAUSDT', 'RUNEUSDT', 'ALGOUSDT',
                        'COREUSDT', 'FLOWUSDT', 'BSVUSDT', 'SEIUSDT', 'EGLDUSDT',
                        'XTZUSDT', 'NEOUSDT', 'CFXUSDT', 'MINAUSDT', 'IOTAUSDT',
                        'LUNCUSDT', 'CKBUSDT', 'IOTXUSDT', 'KSMUSDT', 'DYMUSDT',
                        'ZILUSDT', 'CELOUSDT', 'QTUMUSDT', 'RVNUSDT', 'ONEUSDT',
                        'ONTUSDT', 'ICXUSDT', 'BBUSDT', 'COTIUSDT', 'SXPUSDT',
                        'LSKUSDT', 'NTRNUSDT', 'TAIKOUSDT', 'WAXPUSDT', 'IOSTUSDT

In [119]:
# Upload or Update system-config file
endpoint = "upload/system-config"
url = root_url + endpoint

system_config_name = "futures_config"
base_path = "/Users/seongmin/Desktop/neomatrix_trading/futures"
file_path = os.path.join(base_path, system_config_name + ".py")

params = {"tradeType": "futures", "overwrite": True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'Futures' System-Config Updated Successfully!"}


In [120]:
# Check system-config file upload
endpoint = "upload/check/system-config"
url = root_url + endpoint

params = {"tradeType": "futures"}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# ==========================
# Required System Settings
# ==========================
system_config = {
    "data_apikey": "a71eaf04-802f-40be-93c2-5bee2548f4db",
    "tz_str": "Asia/Seoul",
    "timeframe": "1min",
    "orderType": "market",
    "productType": "usdt-futures",
    "posMode": "hedge_mode",
    "marginMode": "crossed",
    "marginCoin": "usdt",
    "leverage": 5,
    "trading_hours": 6,
    "total_allocation": 0.4,
}


# ==========================
# Rebalancing Trade Parameters
# ==========================
rebalancing_config = {
    "rebalancing_interval_hours": 1, # Rebalancing cycle (hours)
}



In [123]:
# run-system
endpoint = 'command/run-system'
url = root_url + endpoint

data = {
    "tradeType": "futures",
    "strategy_name": strategy_name,
    "trade_mode": "rebalancing",
    "trade_env": "live"
}

response = requests.post(url, headers=headers, json=data)

try:
    session_id = response.json()['session_id']
    print('session_id :',response.json()['session_id'])
    print('dashboard :',response.json()['dashboard_url'])
except:
    print('Error :',response.json()['message']['message'])

    session_id = response.json()['message']['session_id']
    print('session_id id :',session_id)

session_id : 176fff23
dashboard : https://aifapbt.fin.cloud.ainode.ai/dashboard/live_futures_trading_dashbaord.html


In [122]:
# Terminate process
# You must execute terminate to force liquidation of the current process.
endpoint = 'command/terminate'
url = root_url + endpoint

params = {"session_id": f"{session_id}", "tradeType": 'futures'}

response = requests.get(url, headers=headers, params=params)
print(response.json())

{'message': 'Session c7debc40 is terminated.'}


# Check

In [73]:
import pandas as pd
import requests

def all_positions(USER_KEY: str, productType: str, marginCoin: str) -> pd.DataFrame:
    url = "https://bitgettrader.fin.cloud.ainode.ai/futures/position/all-positions"
  
    headers = {
        "API-KEY": USER_KEY,
        "Target-Email": None
    }

    json = {
        "productType": productType,
        'marginCoin': marginCoin.upper()
    }

    response = requests.post(url, json=json, headers=headers)

    data = response.json()['data']
    df = pd.DataFrame(data)
    return df


In [95]:
import pandas as pd
import requests
from typing import Optional

def flash_close_position(USER_KEY: str, symbol: Optional[str], productType: str, holdSide: Optional[str]) -> pd.DataFrame:
    url = "https://bitgettrader.fin.cloud.ainode.ai/futures/trade/close-position"

    headers = {
        "API-KEY": USER_KEY,
        "Target-Email": None
    }

    json = {
        "symbol": symbol,
        "productType": productType,
        "holdSide": holdSide
    }

    response = requests.post(url=url, json=json, headers=headers)
    data = response.json()['data']

    rows = []
    # Success list processing
    for item in data.get('successList', []):
        orderId = item.get('orderId', '')
        clientOid = item.get('clientOid', '')
        result = "success"
        rows.append([orderId, clientOid, result, "", ""])

    # Failure list processing
    for item in data.get('failureList', []):
        orderId = item.get('orderId', '')
        clientOid = item.get('clientOid', '')
        result = "failure"
        errorMsg = item.get('errorMsg', '')
        errorCode = item.get('errorCode', '')
        rows.append([orderId, clientOid, result, errorMsg, errorCode])

    df = pd.DataFrame(rows, columns=["orderId", "clientOid", "result", "errorMsg", "errorCode"])
    return df


In [96]:
df_all_position = all_positions(USER_KEY, 'usdt-futures', 'USDT')
df_all_position

,marginCoin,symbol,holdSide,openDelegateSize,marginSize,available,locked,total,leverage,achievedProfits,...,deductedFee,grant,assetMode,autoMargin,takeProfit,stopLoss,takeProfitId,stopLossId,cTime,uTime
0,USDT,XRPUSDT,short,0,11.5264,20,0,20,5,0,...,0.0230528,,single,off,,,,,1759136520904,1759136520904
1,USDT,DOGEUSDT,long,0,11.718,252,0,252,5,0,...,0.023436,,single,off,,,,,1759136521047,1759136521047
